# Crumple Video - Blender XPBD Paper Animation

Генерация видеоанонса с эффектом раскрывающейся бумаги.

**Особенности:**
- Полная Blender XPBD физика на CPU
- Обложка с датами и типографикой
- Аудиотрек The xx - Intro (старт с 1:10)

**Версия v2.0 - с полными XPBD скриптами**

In [ ]:
NOTEBOOK_VERSION = 'v2.1-blender36'
print(f'🚀 Engine {NOTEBOOK_VERSION}')

# ==========================================
# 1. УСТАНОВКА ЗАВИСИМОСТЕЙ (Blender 3.6)
# ==========================================
import warnings
warnings.filterwarnings("ignore")

print("⏳ Installing dependencies...", flush=True)

# Install FFmpeg from apt
!apt-get update -qq && apt-get install -y -qq ffmpeg libxrender1 libxi6 libxkbcommon-x11-0 > /dev/null 2>&1

# Install Blender 3.6 LTS from tarball (compatible with XPBD script)
import os
if not os.path.exists('/opt/blender-3.6'):
    print('📦 Downloading Blender 3.6 LTS...', flush=True)
    !wget -q https://download.blender.org/release/Blender3.6/blender-3.6.16-linux-x64.tar.xz -O /tmp/blender.tar.xz
    !mkdir -p /opt && tar -xf /tmp/blender.tar.xz -C /opt
    !mv /opt/blender-3.6.* /opt/blender-3.6
    !ln -sf /opt/blender-3.6/blender /usr/local/bin/blender
    !rm /tmp/blender.tar.xz
else:
    print('✅ Blender 3.6 already installed')

# Verify Blender version
!blender --version | head -2

!pip install opencv-python numpy pillow requests > /dev/null

import os
import sys
import json
import glob
import shutil
import subprocess
from pathlib import Path

import numpy as np
import cv2
from PIL import Image, ImageFont, ImageDraw

def log(msg):
    print(msg, flush=True)

log("✅ Dependencies ready")


In [ ]:
# ==========================================
# 2. КОНФИГУРАЦИЯ
# ==========================================
KAGGLE_INPUT_ROOT = "/kaggle/input"
WORKING_DIR = Path("/kaggle/working")

# Ищем папки сессии
session_dirs = sorted(glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "video-afisha-session-*")), reverse=True)
crumple_test_dirs = sorted(glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "crumple-video-test*")), reverse=True)

if session_dirs:
    SOURCE_FOLDER = Path(session_dirs[0])
    log(f"✅ DETECTED BOT SESSION: {SOURCE_FOLDER}")
elif crumple_test_dirs:
    SOURCE_FOLDER = Path(crumple_test_dirs[0])
    log(f"✅ DETECTED TEST DATASET: {SOURCE_FOLDER}")
elif os.path.exists(os.path.join(KAGGLE_INPUT_ROOT, "afisha-dataset-2")):
    SOURCE_FOLDER = Path(os.path.join(KAGGLE_INPUT_ROOT, "afisha-dataset-2"))
    log(f"⚠️ Using legacy dataset: {SOURCE_FOLDER}")
else:
    all_dirs = [d for d in glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "*")) if os.path.isdir(d)]
    SOURCE_FOLDER = Path(all_dirs[0]) if all_dirs else Path("/kaggle/input/afisha-dataset-2")
    log(f"⚠️ Fallback: {SOURCE_FOLDER}")

# Robust payload detection
print("DEBUG: Searching for payload.json in", KAGGLE_INPUT_ROOT)
payloads = list(Path(KAGGLE_INPUT_ROOT).rglob("payload.json"))
if payloads:
    SOURCE_FOLDER = payloads[0].parent
    log(f"✅ Found payload.json at {payloads[0]}, setting SOURCE_FOLDER to {SOURCE_FOLDER}")
else:
    log("⚠️ payload.json not found via rglob, listing input dir:")
    for root, dirs, files in os.walk(KAGGLE_INPUT_ROOT):
        for f in files:
            print(os.path.join(root, f))

# Параметры видео
W, H = 1080, 1920  # Вертикальный формат
FPS = 24

# Аудио
AUDIO_START_SEC = 70  # 1:10 от начала трека
AUDIO_FILENAME = "The_xx_-_Intro.mp3"

# Blender params - больше кадров для плавности
FRAME_START = 1
FRAME_END = 64
BLENDER_SAMPLES = 48  # Более высокое качество
UNFOLD_DELAY = 0.15
UNFOLD_GAMMA = 0.9
SIMILARITY_THRESHOLD = 0.998

WORKING_DIR.mkdir(parents=True, exist_ok=True)
log("✅ Config ready")

In [ ]:
import sys
import os
import shutil
from pathlib import Path


import sys
import os
import shutil
from pathlib import Path

# Fix: Copy assets from read-only input to working dir to allow proper importing
possible_paths = [
    Path("/kaggle/input/video-announce-assets"),
    Path("/kaggle/input/video-announce-assets/video-announce-assets"),
]

dataset_dir = None
for p in possible_paths:
    if p.exists():
        dataset_dir = p
        break

working_dir = Path("/kaggle/working")
if dataset_dir:
    print(f"✅ Found assets dataset at {dataset_dir}")
    
    # Copy pattern_preview.py
    pp_src = dataset_dir / "pattern_preview.py"
    if pp_src.exists():
        shutil.copy2(pp_src, working_dir / "pattern_preview.py")
        print("✅ Copied pattern_preview.py to working dir")
    else:
        print(f"⚠️ pattern_preview.py not found in {dataset_dir}")

    # Copy assets folder
    assets_src = dataset_dir / "assets"
    assets_dst = working_dir / "assets"
    
    if assets_src.exists():
        if assets_dst.exists():
            shutil.rmtree(assets_dst)
        shutil.copytree(assets_src, assets_dst)
        print("✅ Copied assets folder to working dir")
    else:
        # Fallback for loose files
        os.makedirs(assets_dst, exist_ok=True)
        copied_any = False
        for ext in ["*.ttf", "*.otf"]:
            for f in dataset_dir.glob(ext):
                shutil.copy2(f, assets_dst)
                copied_any = True
        if copied_any:
            print("✅ Copied loose font files to assets/ folder")
        else:
            print("⚠️ assets folder/files not found in dataset")
else:
    print("⚠️ Dataset video-announce-assets NOT FOUND")

# Now import from working dir
try:
    import pattern_preview
    print(f"✅ Imported pattern_preview from {pattern_preview.__file__}")
except ImportError as e:
    print(f"⚠️ Could not import pattern_preview: {e}")


from pathlib import Path

def _normalize_pattern(pattern: str) -> str:
    if hasattr(pattern_preview, 'ALL_PATTERNS') and pattern in pattern_preview.ALL_PATTERNS:
        return pattern
    return getattr(pattern_preview, 'PATTERN_STICKER', 'STICKER')

def generate_intro_image(
    pattern: str,
    intro_text: str,
    cities: list[str] | str | None,
    output_path: Path,
) -> Path:
    # Generate intro image using pattern_preview assets.
    normalized = _normalize_pattern(str(pattern or ""))
    if isinstance(cities, str):
        cities_str = cities
    else:
        cities_str = ", ".join([c for c in (cities or []) if c]) or None
    intro_text = intro_text or "АФИША"

    image_bytes = pattern_preview.generate_intro_preview(normalized, intro_text, cities_str)

    out_path = Path(output_path)
    out_path.write_bytes(image_bytes)
    print(f"✅ Saved intro preview: {out_path}")
    return out_path


In [ ]:
# ==========================================
# 4. BLENDER XPBD SCRIPT (встроенный)
# ==========================================
import base64

# Полный blender_xpbd_paper.py (780 строк) закодирован в base64
BLENDER_SCRIPT_B64 = """..."""

# Декодируем и сохраняем скрипт
BLENDER_SCRIPT_PATH = WORKING_DIR / "blender_xpbd_paper.py"
BLENDER_SCRIPT_PATH.write_bytes(base64.b64decode(BLENDER_SCRIPT_B64))
log(f"✅ Embedded Blender XPBD script written: {BLENDER_SCRIPT_PATH} ({BLENDER_SCRIPT_PATH.stat().st_size} bytes)")

def run_blender_render(poster_path: Path, out_dir: Path) -> bool:
    """Run Blender in background to render frames."""
    if not BLENDER_SCRIPT_PATH.exists():
        log(f"❌ Blender script not found: {BLENDER_SCRIPT_PATH}")
        return False
    
    out_dir.mkdir(parents=True, exist_ok=True)
    
    cmd = [
        "blender", "-b", "--factory-startup", "-noaudio",
        "-P", str(BLENDER_SCRIPT_PATH),
        "--",
        "--poster", str(poster_path),
        "--out_dir", str(out_dir),
        "--frame_start", str(FRAME_START),
        "--frame_end", str(FRAME_END),
        "--unfold_delay", str(UNFOLD_DELAY),
        "--unfold_gamma", str(UNFOLD_GAMMA),
        "--samples", str(BLENDER_SAMPLES)
    ]
    log(f"🎬 Rendering: {poster_path.name}")
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=1800)
        if result.returncode != 0:
            log(f"❌ Blender error (exit {result.returncode}):")
            log(result.stderr[:1000] if result.stderr else "No stderr")
            return False
    except subprocess.TimeoutExpired:
        log("❌ Blender timeout (30 min)")
        return False
    except Exception as e:
        log(f"❌ Blender exception: {e}")
        return False
    
    # Проверяем что кадры созданы
    frames = list(out_dir.glob("frame_*.png"))
    if not frames:
        log("❌ No frames generated")
        return False
    
    # Проверяем размер первого кадра
    first_frame = sorted(frames)[0]
    frame_size = first_frame.stat().st_size
    if frame_size < 10000:
        log(f"⚠️ Frame too small ({frame_size} bytes)")
    
    log(f"✅ Done: {len(frames)} frames, first={frame_size/1024:.1f}KB")
    return True

log("✅ Blender runner ready")


In [ ]:
# ==========================================
# 5. SHOWREEL ASSEMBLY (из make_showreel_audio.py)
# ==========================================
from dataclasses import dataclass
import math

def blend(a, b, t: float):
    """Blend two frames with weight t (0.0 = a, 1.0 = b)"""
    t = float(max(0.0, min(1.0, t)))
    return cv2.addWeighted(a, 1.0 - t, b, t, 0.0)

def ease_linear(x: float) -> float:
    return x

def ease_in_quad(x: float) -> float:
    return x * x

def ease_out_quad(x: float) -> float:
    return 1.0 - (1.0 - x) * (1.0 - x)

def ease_in_out_cubic(x: float) -> float:
    if x < 0.5:
        return 4.0 * x * x * x
    return 1.0 - pow(-2.0 * x + 2.0, 3.0) / 2.0

def ease_out_expo(x: float) -> float:
    return 1.0 if x >= 1.0 else 1.0 - pow(2, -10 * x)

def ease_in_out_back(x: float) -> float:
    c1 = 1.70158
    c2 = c1 * 1.525
    if x < 0.5:
        return (pow(2 * x, 2) * ((c2 + 1) * 2 * x - c2)) / 2
    return (pow(2 * x - 2, 2) * ((c2 + 1) * (x * 2 - 2) + c2) + 2) / 2

EASING = {
    "linear": ease_linear,
    "ease_in_quad": ease_in_quad,
    "ease_out_quad": ease_out_quad,
    "ease_in_out_cubic": ease_in_out_cubic,
    "ease_out_expo": ease_out_expo,
    "ease_in_out_back": ease_in_out_back,
}

def render_motion(base: list, length: int, easing_name: str, reverse: bool = False, frame_interp: bool = True) -> list:
    """Render motion with proper easing and frame interpolation."""
    if length <= 0 or not base:
        return []
    if len(base) == 1:
        return [base[0]] * int(length)
    
    ease = EASING.get(easing_name, ease_in_out_cubic)
    out = []
    n = max(1, int(length))
    
    for i in range(n):
        t = 0.0 if n == 1 else (i / (n - 1))
        s = float(max(0.0, min(1.0, ease(t)))) * (len(base) - 1)
        if reverse:
            s = (len(base) - 1) - s
        
        if not frame_interp:
            idx = int(round(s))
            out.append(base[idx])
        else:
            i0 = int(math.floor(s))
            i1 = min(len(base) - 1, i0 + 1)
            alpha = float(s - i0)
            out.append(blend(base[i0], base[i1], alpha))
    
    return out

# Timing parameters from original make_showreel_audio.py
UNFOLD_LEN = 24
FOLD_LEN = 24
HOLD_FLAT = 2
HOLD_BALL = 2
FRAME_INTERP = True

@dataclass
class SegmentPlan:
    stem: str
    unfold_len: int = UNFOLD_LEN
    fold_len: int = FOLD_LEN
    hold_flat: int = HOLD_FLAT
    hold_ball: int = HOLD_BALL
    easing_unfold: str = "ease_in_out_back"
    easing_fold: str = "ease_out_quad"

def composite_rgba_on_bg(rgba_frame, bg_color=(20, 20, 25)):
    """Composite RGBA frame onto dark gradient background."""
    h, w = rgba_frame.shape[:2]
    bg = np.zeros((h, w, 3), dtype=np.uint8)
    for y in range(h):
        factor = 0.8 + 0.2 * (y / h)
        bg[y, :] = [int(c * factor) for c in bg_color]
    
    if rgba_frame.shape[2] == 4:
        alpha = rgba_frame[:, :, 3:4].astype(np.float32) / 255.0
        rgb = rgba_frame[:, :, :3].astype(np.float32)
        result = (rgb * alpha + bg.astype(np.float32) * (1 - alpha))
        return result.astype(np.uint8)
    else:
        return rgba_frame[:, :, :3]

def _prep_similarity(frame):
    small = cv2.resize(frame, (96, 96), interpolation=cv2.INTER_AREA)
    return cv2.cvtColor(small, cv2.COLOR_RGB2GRAY)

def frame_similarity(a, b) -> float:
    if a is None or b is None:
        return 0.0
    ga = _prep_similarity(a)
    gb = _prep_similarity(b)
    if ga.shape != gb.shape:
        return 0.0
    diff = cv2.absdiff(ga, gb)
    mean = float(diff.mean()) / 255.0
    return 1.0 - mean

def load_segment_frames(
    frames_dir: Path,
    frame_start: int,
    frame_end: int,
    similarity_threshold: float = SIMILARITY_THRESHOLD,
) -> list:
    """Load and composite Blender frames for a segment."""
    frames = []
    last = None
    dropped = 0
    for fn in range(frame_start, frame_end + 1):
        frame_path = frames_dir / f"frame_{fn:04d}.png"
        if not frame_path.exists():
            continue
        img = cv2.imread(str(frame_path), cv2.IMREAD_UNCHANGED)
        if img is None:
            continue
        composited = composite_rgba_on_bg(img)
        rgb = cv2.cvtColor(composited, cv2.COLOR_BGR2RGB)
        if similarity_threshold is not None and last is not None:
            if frame_similarity(last, rgb) >= similarity_threshold:
                dropped += 1
                continue
        frames.append(rgb)
        last = rgb
    if dropped:
        log(f"⚠️ Dropped {dropped} near-duplicate frames from {frames_dir.name}")
    return frames

log("✅ Showreel functions ready")



In [ ]:
# ==========================================
# 6. AUDIO & VIDEO ASSEMBLY
# ==========================================

def add_audio(video_path: Path, audio_path: Path, output_path: Path, start_sec: float = 70) -> bool:
    """Mix audio starting at specified timestamp with video."""
    if not audio_path.exists():
        log(f"⚠️ Audio not found: {audio_path}")
        shutil.copy(video_path, output_path)
        return True
    
    cmd = [
        "ffmpeg", "-y",
        "-ss", str(float(start_sec)),
        "-i", str(audio_path),
        "-i", str(video_path),
        "-shortest",
        "-c:v", "copy",
        "-c:a", "aac",
        "-b:a", "192k",
        str(output_path)
    ]
    result = subprocess.run(cmd, capture_output=True)
    if result.returncode == 0:
        log(f"✅ Audio added: {output_path}")
        return True
    else:
        log(f"❌ FFmpeg audio error")
        return False

def render_video_from_frames(frames_dir: Path, output_path: Path, fps: int = 24) -> bool:
    """Use FFmpeg to assemble frames into video."""
    cmd = [
        "ffmpeg", "-y",
        "-framerate", str(fps),
        "-i", str(frames_dir / "frame_%06d.png"),
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        "-crf", "18",
        str(output_path)
    ]
    result = subprocess.run(cmd, capture_output=True)
    if result.returncode == 0:
        log(f"✅ Video rendered: {output_path} ({output_path.stat().st_size / 1024:.1f} KB)")
        return True
    else:
        log(f"❌ FFmpeg video error")
        return False

log("✅ Audio/Video functions ready")

In [ ]:
# ==========================================
# 7. ОСНОВНОЙ ПАЙПЛАЙН (Updated for Outro & Cities)
# ==========================================


def _resolve_image_path(filename: str):
    p = SOURCE_FOLDER / filename
    if p.exists(): return p
    p = Path(KAGGLE_INPUT_ROOT) / filename
    if p.exists(): return p
    for base in [SOURCE_FOLDER, Path(KAGGLE_INPUT_ROOT)]:
        try:
            matches = list(base.rglob(filename))
            if matches: return matches[0]
        except: pass
    return None

def main_pipeline():
    """Full pipeline."""
    
    payload_path = SOURCE_FOLDER / "payload.json"
    
    # Fallback for missing payload.json
    if not payload_path.exists():
        log(f"⚠️ payload.json not found at {payload_path}. Generatng fallback.")
        payload_path = WORKING_DIR / "payload.json"
        fallback_scenes = []
        # Find images in source folder
        for ext in ["*.jpg", "*.png"]:
            for img in SOURCE_FOLDER.glob(ext):
                fallback_scenes.append({"image": img.name, "text": f"Fallback {img.stem}"})
        # Sort and take top 4
        fallback_scenes.sort(key=lambda x: x["image"])
        fallback_data = {"scenes": fallback_scenes[:4]}
        with open(payload_path, "w") as f:
            json.dump(fallback_data, f)
        log(f"✅ Generated fallback payload at {payload_path} with {len(fallback_scenes)} scenes")
    
    try:
        payload = json.loads(payload_path.read_text())
    except Exception as e:
        log(f"❌ Failed to read payload: {e}")
        return False

    scenes = payload.get("scenes", [])
    
    intro = payload.get("intro") or {}
    cover = payload.get("cover") or {}
    intro_text = intro.get("text") or cover.get("title") or "АФИША"
    intro_pattern = intro.get("pattern") or pattern_preview.PATTERN_STICKER
    intro_cities = intro.get("cities") or cover.get("cities") or []
    intro_path = WORKING_DIR / "intro.png"
    
    # Generate Intro via pattern_preview
    generate_intro_image(intro_pattern, intro_text, intro_cities, intro_path)

    posters = [intro_path]
    
    # Collect scene posters
    for i, scene in enumerate(scenes[:12]): # Max 12 as per user spec
        # Try to find poster files
        # The payload might specify image filename directly
        img_name = scene.get("image")
        found = None
        if img_name:
             found = _resolve_image_path(img_name)
        
        if not found:
            # Fallback to posterN naming
            for ext in [".jpg", ".png", ".jpeg"]:
                local = SOURCE_FOLDER / f"poster{i+1}{ext}"
                if local.exists():
                    found = local
                    break
        
        if found:
            posters.append(found)
    


    # Check for Final scene
    final_img = _resolve_image_path("Final.png")
    if final_img:
        posters.append(final_img)
        log(f"✅ Appended final scene: {final_img}")
    else:
        log("⚠️ Final.png not found; skipping final scene")
        
    log(f"Posters to render: {len(posters)}")
    
    # Build segments
    segments = []
    
    # 1. Intro (Cover) - Starts Flat, Crumples UP (Fold)
    # We want Intro to just be visible then crumple away?
    # Or start crumpled and uncrumple?
    # If stem='frames_0', run_blender_render produced 1..64 (1=Crumpled, 64=Flat).
    # Standard flow: Unfold (Ball->Flat) -> Show -> Fold (Flat->Ball).
    # Cover usually: Starts Flat (no unfold), Shows, then Folds (to transition to next).
    segments.append(SegmentPlan(
        stem="frames_0",
        unfold_len=0,     # Start Flat
        fold_len=24,      # Crumple away
        hold_flat=48 if is_last else 2,
        hold_ball=2,
        easing_unfold="linear",
        easing_fold="ease_out_quad"
    ))
    
    # 2. Main Scenes + Outro
    # Iterate posters starting from index 1
    for i in range(1, len(posters)):
        is_last = (i == len(posters) - 1)
        
        # If this is the LAST poster (Outro/Final), we want it to Stay Open.
        # Unfold -> Hold Flat (Long) -> No Fold
        
        segments.append(SegmentPlan(
            stem=f"frames_{i}",
            unfold_len=0 if is_last else 28,
            fold_len=0 if is_last else 28,      # No fold for Outro
            hold_flat=48 if is_last else 2,     # Hold longer for Outro
            hold_ball=0 if is_last else 2,
            easing_unfold="ease_in_out_back",
            easing_fold="ease_in_out_back"
        ))
    
    # Render Blender frames
    for i, poster in enumerate(posters):
        frames_dir = WORKING_DIR / f"frames_{i}"
        # Check if already rendered (re-entrancy)
        if frames_dir.exists() and len(list(frames_dir.glob("*.png"))) >= FRAME_END - FRAME_START:
            log(f"Skipping render for {poster.name} (cached)")
            continue
            
        success = run_blender_render(poster, frames_dir)
        if not success:
            log(f"Skipping poster {i}")
    
    # Assemble sequence
    log("Assembling showreel...")
    sequence = []
    
    for seg in segments:
        frames_dir = WORKING_DIR / seg.stem
        frames = load_segment_frames(frames_dir, FRAME_START, FRAME_END)
        
        if not frames:
            log(f"No frames for {seg.stem}")
            continue
        
        ball = frames[0]
        flat = frames[-1]
        
        # Unfold (Ball -> Flat)
        if seg.unfold_len > 0:
            unfold = render_motion(frames, seg.unfold_len, seg.easing_unfold, False, True)
            sequence.extend(unfold)
        
        # Hold Flat
        for _ in range(seg.hold_flat):
            sequence.append(flat)
        
        # Fold (Flat -> Ball)
        if seg.fold_len > 0:
            fold = render_motion(frames, seg.fold_len, seg.easing_fold, True, True)
            sequence.extend(fold)
        
        # Hold Ball
        for _ in range(seg.hold_ball):
            sequence.append(ball)
    
    log(f"Total frames: {len(sequence)}")
    
    if not sequence:
        log("FATAL: No frames!")
        return False
    
    seq_dir = WORKING_DIR / "sequence"
    if seq_dir.exists():
        import shutil
        shutil.rmtree(seq_dir)
    seq_dir.mkdir(exist_ok=True)
    
    for i, frame in enumerate(sequence):
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        cv2.imwrite(str(seq_dir / f"frame_{i+1:06d}.png"), frame_bgr)
    
    video_path = WORKING_DIR / "crumple_video.mp4"
    render_video_from_frames(seq_dir, video_path, fps=24)
    
    final_path = WORKING_DIR / "crumple_video_final.mp4"
    audio_path = list(SOURCE_FOLDER.glob("*.mp3"))
    if audio_path:
        add_audio(video_path, audio_path[0], final_path)
    else:
        import shutil
        shutil.copy(video_path, final_path)
    
    if final_path.exists():
        size_kb = final_path.stat().st_size / 1024
        log(f"Final video: {size_kb:.1f} KB")
        if size_kb < 500:
            log("FAIL: Video too small!")
            return False
        return True
    
    return False

log("=" * 50)
log("Starting CrumpleVideo Pipeline")
log("=" * 50)

try:
    success = main_pipeline()
except Exception as e:
    import traceback
    traceback.print_exc()
    success = False

if success:
    log("Pipeline completed!")
else:
    log("Pipeline failed!")
